In [ ]:
# ─── Standard Library Imports ─────────────────────────────
import os  # Operating system interaction
import sys  # Access to system-specific parameters and functions
import json  # Reading and writing JSON configuration
import zipfile  # Handling ZIP file creation

# ─── Third-party Imports ──────────────────────────────────
import cv2  # OpenCV for image processing
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and tables
import openpyxl  # Excel file I/O
import matplotlib.pyplot as plt  # Plotting
import matplotlib  # Matplotlib config
matplotlib.use("Qt5Agg")  # Use Qt5Agg backend for GUI support

# ─── PyQt5 GUI Components ────────────────────────────────
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QLineEdit, QPushButton,
    QVBoxLayout, QHBoxLayout, QFileDialog, QMessageBox,
    QTextEdit, QInputDialog, QComboBox
)

# ─── Image I/O ───────────────────────────────────────────
import imageio.v2 as imageio  # Image reading/writing (legacy v2 API)

# ─── Skimage Modules for Image Processing ────────────────
from skimage.measure import label, regionprops  # Region labeling
from skimage.filters import threshold_li, threshold_otsu, threshold_isodata, sobel, threshold_local,gabor  # Threshold methods
from skimage import data, filters, measure, exposure, morphology, segmentation, feature, io # Generic image ops
from skimage.color import rgb2gray, label2rgb  # Convert RGB to grayscale
from skimage.morphology import (
    opening, remove_small_objects, remove_small_holes, disk, dilation, closing, erosion
)  # Morphological ops
from skimage import exposure, color  # Image enhancement and color ops
from skimage.feature import peak_local_max  # Peak detection
from skimage.segmentation import (
    morphological_chan_vese, slic, active_contour,
    watershed, random_walker
)  # Various segmentation algorithms
from skimage.io import imread  # Image reading
from skimage.transform import resize  # Image resizing
from skimage import draw  # Drawing shapes
import skimage.exposure
from skimage.exposure import rescale_intensity
from skimage.util import img_as_ubyte

# ─── SciPy for Advanced Processing ───────────────────────
import scipy.ndimage as ndi  # Multidimensional processing
from scipy import ndimage as ndi
from scipy.ndimage import distance_transform_edt, label as ndi_label  # Distance transforms and labeling
from scipy import ndimage  # General ndimage support
from scipy.signal import find_peaks  # Signal peak detection

# ─── Machine Learning ─────────────────────────────────────
from sklearn.cluster import KMeans  # Clustering (e.g., for region grouping)

# ─── Excel Writing ───────────────────────────────────────
from xlsxwriter import Workbook  # Advanced Excel writing

# ─── Qt Event Processing ─────────────────────────────────
QApplication.processEvents()  # Process any pending GUI events

# ─── Threading & Event Control ───────────────────────────
from threading import Event  # Used to signal stopping of processing

# ─── Utilities ────────────────────────────────────────────
from collections import defaultdict  # Dictionary that creates default values automatically

import gc

# ─────────────────────────────────────────────────────────
# GUI Application Class for Image Processing
class ImageProcessingApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()  # Set up GUI layout

        # Initialize folder paths and control flags
        self.input_folder = ""
        self.output_folder = ""
        self.processing_active = False  # Track if a process is currently running
        self.stop_event = Event()  # Event to handle stop signal

    def initUI(self):
        # Create the GUI layout
        layout = QVBoxLayout()

        # Labels for folder selection display
        self.input_label = QLabel("Input Folder: Not selected")
        self.output_label = QLabel("Output Folder: Not selected")

        # Buttons for actions and controls
        self.input_button = QPushButton("Select Input Folder")
        self.output_button = QPushButton("Select Output Folder")
        self.process_button = QPushButton("Detector GRAYSCALE")
        self.process_button_2 = QPushButton("Detector MAGENTA")
        self.stop_button = QPushButton("Stop Processing")
        self.restart_button = QPushButton("Restart Processing")

        # Log output window
        self.log_output = QTextEdit()
        self.log_output.setReadOnly(True)

        # Connect button actions to their corresponding methods
        self.input_button.clicked.connect(self.select_input_folder)
        self.output_button.clicked.connect(self.select_output_folder)
        self.process_button.clicked.connect(self.start_processing)
        self.process_button_2.clicked.connect(self.start_processing_2)
        self.stop_button.clicked.connect(self.stop_processing)
        self.restart_button.clicked.connect(self.restart_processing)

        # Add widgets to the GUI layout
        layout.addWidget(self.input_label)
        layout.addWidget(self.input_button)
        layout.addWidget(self.output_label)
        layout.addWidget(self.output_button)
        layout.addWidget(self.process_button)
        layout.addWidget(self.process_button_2)
        layout.addWidget(self.log_output)
        layout.addWidget(self.stop_button)
        layout.addWidget(self.restart_button)

        # Finalize window settings
        self.setLayout(layout)
        self.setWindowTitle("Batch Image Processing")
        self.resize(500, 400)

    def log(self, message):
        # Append a log message to the log output display (likely a QTextEdit or QListWidget)
        self.log_output.append(message)

    def on_custom_um_entered(self):
        # Handle user entering a custom µm value in the combo box
        text = self.known_um_combo.currentText().strip()
    
        # If the entered text is not already in the combo box, add it
        if text not in [self.known_um_combo.itemText(i) for i in range(self.known_um_combo.count())]:
            self.known_um_combo.addItem(text)

    def select_input_folder(self):
        # Prompt user to select a folder for BF (Brightfield) images
        self.input_folder = QFileDialog.getExistingDirectory(self, "Select Input Folder")
        self.input_label.setText(f"BF Folder: {self.input_folder}")

    def select_output_folder(self):
        # Prompt user to select a folder to save outputs
        self.output_folder = QFileDialog.getExistingDirectory(self, "Select Output Folder")
        self.output_label.setText(f"Output Folder: {self.output_folder}")

    def stop_processing(self):
        # Set the stop event flag to signal that processing should stop
        self.stop_event.set()
        self.log("Stopping process...")

    def restart_processing(self):
        # Stop current process and then start Script 3 again
        self.stop_processing()
        self.log("Restarting processing...")
        self.start_processing_3()

    def start_processing(self):
        # Flag to indicate that processing is active
        self.processing_active = True

        # Reset the stop event in case it was triggered during a previous run
        self.stop_event.clear()

        # Validate that all necessary folders (BF, PL, and Output) have been selected
        if not self.input_folder or not self.output_folder:
            self.log("Please select all folders before starting.")
            return

        # Create the output directory if it doesn't already exist
        os.makedirs(self.output_folder, exist_ok=True)

        # Collect and sort all .tif files in both BF and PL folders
        input_file = sorted([f for f in os.listdir(self.input_folder) if f.endswith('.png')])
        #input_file = sorted([f for f in os.listdir(self.input_folder) if f.endswith('.jpeg')])

        # List to keep track of output files generated during processing
        all_output_files = []

        # Batch process each image file
        for file_name in input_file:
            print(f"Processing: {file_name}")

            # Allow user to stop processing midway
            if self.stop_event.is_set():
                self.log("Processing stopped.")
                return

            self.log(f"Processing {file_name}...")

            # Load image
            input_image_path = os.path.join(self.input_folder, file_name)
            imageA = cv2.imread(input_image_path)

            # Convert BF image to grayscale
            grayA = rgb2gray(imageA)

            # Convert to uint8, range [0,255]
            grayA = (grayA * 255).astype(np.uint8)

            # Define Gabor orientations in radians (0°, 45°, 90°)
            orientations = [0, np.pi/4, np.pi/2]##[0, np.pi/4, np.pi/2]
            frequency = 0.15#0.2

            # Apply Gabor filters and take maximum response
            gabor_responses = []
            for theta in orientations:
                real, _ = gabor(grayA, frequency=frequency, theta=theta)
                gabor_responses.append(real)

            # Combine responses by taking the maximum pixel-wise value
            gabor_combined = np.max(np.stack(gabor_responses, axis=0), axis=0)

            # Normalize to 0-255
            gabor_norm = exposure.rescale_intensity(gabor_combined, out_range=(0, 255)).astype(np.uint8)

            # Threshold using Otsu
            _, binary_A = cv2.threshold(gabor_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            binary_A = morphology.dilation(binary_A, morphology.disk(2))#add
            binary_A = morphology.closing(binary_A, morphology.disk(2))#add
            binary_A = morphology.erosion(binary_A, morphology.disk(2))#add
            binary_A = (binary_A * 255).astype(np.uint8)

            # Save results
            cv2.imwrite("gabor_combined_response.png", gabor_norm)
            cv2.imwrite("gabor_segmented_mask.png", binary_A)

            # Show result
            plt.figure(figsize=(10,5))
            plt.subplot(1,2,1)
            plt.imshow(gabor_norm, cmap='gray')
            plt.title("Gabor Combined Response")
            plt.axis('off')
            plt.pause(0.001)
            QApplication.processEvents()
            plt.close()

            plt.subplot(1,2,2)
            plt.imshow(binary_A)
            plt.title("Segmented")
            plt.axis('off')
            plt.pause(0.001)
            QApplication.processEvents()
            plt.close()
    
            # Label connected regions
            region_labels_A = label(binary_A)
            region_props_A = regionprops(region_labels_A)
            
            # Ensure binary mask matches grayscale shape
            if binary_A.shape != grayA.shape:
                binary_A = resize(binary_A, grayA.shape, order=0, preserve_range=True, anti_aliasing=False)

            # Annotate region labels on binary image
            #overlay_image = cv2.cvtColor((binary_A > 0).astype(np.uint8) * 255, cv2.COLOR_GRAY2BGR)
            #for region in regionprops(region_labels_A):
            #    y, x = region.centroid
            #    label_id = region.label
            #    cv2.putText(overlay_image, str(region.label), (int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            #----------------new-----------------------------------------------------------------------------------------------------
            # --- Color overlay for multi-component segmentation ---
            overlay_colored = label2rgb(region_labels_A, image=imageA, bg_label=0)
            overlay_colored_uint8 = (overlay_colored * 255).astype(np.uint8)

            overlay_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_MultiComponent_Overlay.png")
            cv2.imwrite(overlay_path, overlay_colored_uint8)
            print(f"Saved multi-color segmentation overlay to {overlay_path}")
            all_output_files.append(overlay_path)

            # Optional: annotate regions with labels on overlay
            overlay_image = overlay_colored_uint8.copy()
            for region in region_props_A:
                y, x = region.centroid
                cv2.putText(overlay_image, str(region.label), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (0, 0, 255), 1, cv2.LINE_AA)
            #--------------------------------------------------------------------------------------------------------------------
            # Save annotated segmentation image
            annotated_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_Segmented_Annotated.png")
            cv2.imwrite(annotated_path, overlay_image)
            print(f"Saved annotated image with labels to {annotated_path}")
            all_output_files.append(annotated_path)

            # Create binary mask with only valid detected regions
            filtered_binary_A = np.zeros_like(binary_A)
            for prop in region_props_A:
                if prop.area > 0:
                    min_row, min_col, max_row, max_col = prop.bbox
                    filtered_binary_A[min_row:max_row, min_col:max_col] = (
                        region_labels_A[min_row:max_row, min_col:max_col] == prop.label
                    )
            filtered_binary_A = (filtered_binary_A > 0).astype(np.uint8) * 255

            # --- Save region statistics to Excel ---
            region_area_df = pd.DataFrame({
                "Region_Label": [region.label for region in region_props_A],
                "Region_Area (pixels)": [region.area for region in region_props_A],
            })

            total_area = region_area_df["Region_Area (pixels)"].sum()
            total_detect = region_area_df["Region_Label"].count()

            # Append summary rows
            region_area_df.loc["Total Area"] = ["Total Area", total_area]
            region_area_df.loc["Total"] = ["Total", total_detect]

            # Save region stats to Excel
            region_area_excel_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_Region_Area.xlsx")
            region_area_df.to_excel(region_area_excel_path, index=False)
            print(f"Saved region areas for {file_name} to {region_area_excel_path}")
        
            del grayA, binary_A, region_labels_A, region_props_A, overlay_image, filtered_binary_A
            gc.collect()

        self.log("Processing complete!")
        
        # -----------------------------------------------------
        # Create a ZIP archive with all output histogram and annotated image files
        zip_path = os.path.join(self.output_folder, "All_Images_histograms.zip")
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file_path in all_output_files:
                zipf.write(file_path, arcname=os.path.basename(file_path))
                
        # Remove the original files after archiving
        for file_path in all_output_files:
            if os.path.exists(file_path):
                os.remove(file_path)

    def start_processing_2(self):
        # Flag to indicate that processing is active
        self.processing_active = True

        # Reset the stop event in case it was triggered during a previous run
        self.stop_event.clear()

        # Validate that all necessary folders (BF, PL, and Output) have been selected
        if not self.input_folder or not self.output_folder:
            self.log("Please select all folders before starting.")
            return

        # Create the output directory if it doesn't already exist
        os.makedirs(self.output_folder, exist_ok=True)

        # Collect and sort all .tif files in both BF and PL folders
        input_file = sorted([f for f in os.listdir(self.input_folder) if f.endswith('.png')])
        #input_file = sorted([f for f in os.listdir(self.input_folder) if f.endswith('.jpeg')])

        # List to keep track of output files generated during processing
        all_output_files = []

        # Batch process each image file
        for file_name in input_file:
            print(f"Processing: {file_name}")

            # Allow user to stop processing midway
            if self.stop_event.is_set():
                self.log("Processing stopped.")
                return

            self.log(f"Processing {file_name}...")

            # Load image
            input_image_path = os.path.join(self.input_folder, file_name)
            imageA = cv2.imread(input_image_path)

            # === Parameters ===
            output_binary_path = "binary_mask_gabor_otsu.png"

            # === Extract Magenta Channel ===
            magenta = imageA[..., 0] + imageA[..., 2]  # R + B
            magenta = np.clip(magenta, 0, 255).astype(np.uint8)

            # Define Gabor orientations in radians (0°, 45°, 90°)
            orientations = [0, np.pi/4, np.pi/2]##[0, np.pi/4, np.pi/2]
            frequency = 0.15#0.2

            # Apply Gabor filters and take maximum response
            gabor_responses = []
            for theta in orientations:
                real, _ = gabor(magenta, frequency=frequency, theta=theta)
                gabor_responses.append(real)

            # Combine responses by taking the maximum pixel-wise value
            gabor_combined = np.max(np.stack(gabor_responses, axis=0), axis=0)

            # Normalize to 0-255
            gabor_norm = exposure.rescale_intensity(gabor_combined, out_range=(0, 255)).astype(np.uint8)

            # Threshold using Otsu
            _, binary_A = cv2.threshold(gabor_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            binary_A = morphology.dilation(binary_A, morphology.disk(1))#add#2.5 funciona
            binary_A = morphology.closing(binary_A, morphology.disk(1))#add#2.5 funciona
            binary_A = morphology.erosion(binary_A, morphology.disk(1))#add
            binary_A = (binary_A * 255).astype(np.uint8)

            # Save results
            cv2.imwrite("gabor_combined_response.png", gabor_norm)
            cv2.imwrite("gabor_segmented_mask.png", binary_A)

            # Show result
            plt.figure(figsize=(10,5))
            plt.subplot(1,2,1)
            plt.imshow(gabor_norm, cmap='gray')
            plt.title("Gabor Combined Response")
            plt.axis('off')
            plt.pause(0.001)
            QApplication.processEvents()
            plt.close()

            plt.subplot(1,2,2)
            plt.imshow(binary_A)
            plt.title("Segmented")
            plt.axis('off')
            plt.pause(0.001)
            QApplication.processEvents()
            plt.close()

            # Label connected regions
            region_labels_A = label(binary_A)
            region_props_A = regionprops(region_labels_A)
            
            # Ensure binary mask matches grayscale shape
            if binary_A.shape != magenta.shape:
                binary_A = resize(binary_A, grayA.shape, order=0, preserve_range=True, anti_aliasing=False)

            # Annotate region labels on binary image
            #overlay_image = cv2.cvtColor((binary_A > 0).astype(np.uint8) * 255, cv2.COLOR_GRAY2BGR)
            #for region in regionprops(region_labels_A):
            #    y, x = region.centroid
            #    label_id = region.label
            #    cv2.putText(overlay_image, str(region.label), (int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            #----------------new-----------------------------------------------------------------------------------------------------
            # --- Color overlay for multi-component segmentation ---
            overlay_colored = label2rgb(region_labels_A, image=imageA, bg_label=0)
            overlay_colored_uint8 = (overlay_colored * 255).astype(np.uint8)

            overlay_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_MultiComponent_Overlay.png")
            cv2.imwrite(overlay_path, overlay_colored_uint8)
            print(f"Saved multi-color segmentation overlay to {overlay_path}")
            all_output_files.append(overlay_path)

            # Optional: annotate regions with labels on overlay
            overlay_image = overlay_colored_uint8.copy()
            for region in region_props_A:
                y, x = region.centroid
                cv2.putText(overlay_image, str(region.label), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (0, 0, 255), 1, cv2.LINE_AA)
            #--------------------------------------------------------------------------------------------------------------------
            # Save annotated segmentation image
            annotated_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_Segmented_Annotated.png")
            cv2.imwrite(annotated_path, overlay_image)
            print(f"Saved annotated image with labels to {annotated_path}")
            all_output_files.append(annotated_path)

            # Create binary mask with only valid detected regions
            filtered_binary_A = np.zeros_like(binary_A)
            for prop in region_props_A:
                if prop.area > 0:
                    min_row, min_col, max_row, max_col = prop.bbox
                    filtered_binary_A[min_row:max_row, min_col:max_col] = (
                        region_labels_A[min_row:max_row, min_col:max_col] == prop.label
                    )
            filtered_binary_A = (filtered_binary_A > 0).astype(np.uint8) * 255

            # --- Save region statistics to Excel ---
            region_area_df = pd.DataFrame({
                "Region_Label": [region.label for region in region_props_A],
                "Region_Area (pixels)": [region.area for region in region_props_A],
            })

            total_area = region_area_df["Region_Area (pixels)"].sum()
            total_detect = region_area_df["Region_Label"].count()

            # Append summary rows
            region_area_df.loc["Total Area"] = ["Total Area", total_area]
            region_area_df.loc["Total"] = ["Total", total_detect]

            # Save region stats to Excel
            region_area_excel_path = os.path.join(self.output_folder, f"{os.path.splitext(file_name)[0]}_Region_Area.xlsx")
            region_area_df.to_excel(region_area_excel_path, index=False)
            print(f"Saved region areas for {file_name} to {region_area_excel_path}")
            
            del magenta, binary_A, region_labels_A, region_props_A, overlay_image, filtered_binary_A
            gc.collect()

        self.log("Processing complete!")
        
        # -----------------------------------------------------
        # Create a ZIP archive with all output histogram and annotated image files
        zip_path = os.path.join(self.output_folder, "All_Images_histograms.zip")
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file_path in all_output_files:
                zipf.write(file_path, arcname=os.path.basename(file_path))
                
        # Remove the original files after archiving
        for file_path in all_output_files:
            if os.path.exists(file_path):
                os.remove(file_path)


# Entry point of the application
if __name__ == "__main__":
    # Create a Qt application instance
    app = QApplication(sys.argv)

    # Instantiate the main window (custom image processing GUI)
    window = ImageProcessingApp()

    # Show the main window
    window.show()

    # Execute the Qt event loop and exit the application when it's closed
    sys.exit(app.exec_())

In [ ]:
# Load .czi file
img = "C://Users//nahue//Downloads//Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.tif"

import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from scipy.ndimage import distance_transform_edt, label
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects

# ==== Load 3D Volume ====
# --- Option A: TIFF stack ---
from tifffile import imread

volume_raw = imread(img)  # shape: (Z, C, Y, X)
print("Raw shape:", volume_raw.shape)

# Select first channel (C=0)
volume = volume_raw[:, 0, :, :]  # shape: (Z, Y, X)
print("Fixed shape:", volume.shape)

# # --- Option B: NIfTI file ---
# import nibabel as nib
# nii = nib.load("your_image.nii")
# volume = nii.get_fdata()

print("Loaded volume shape:", volume.shape)  # Should be (Z, Y, X)

# ==== Preprocess ====
# Normalize intensity
volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))

# Otsu threshold
thresh = threshold_otsu(volume)
binary = volume > thresh

# Optional: Remove small noise
binary = remove_small_objects(binary, min_size=200)

# ==== Watershed Segmentation ====
# Compute distance map
distance = distance_transform_edt(binary)

# Detect local maxima
coords = peak_local_max(distance, labels=binary, footprint=np.ones((3, 3, 3)), exclude_border=False)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = label(mask)

# Apply 3D watershed
labels_ws = watershed(-distance, markers, mask=binary)

# ==== Visualize One Slice ====
z = volume.shape[0] // 2
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(volume[z], cmap='gray')
plt.title('Original (Z slice)')
plt.subplot(1, 2, 2)
plt.imshow(labels_ws[z], cmap='nipy_spectral')
plt.title('Watershed segmentation')
plt.show()

#import imageio

#frames = [(labels_ws[z] * 10).astype(np.uint8) for z in range(volume.shape[0])]
#imageio.mimsave("segmentation_z_stack.gif", frames, duration=0.1)

import napari

viewer = napari.Viewer()
viewer.add_image(volume, name='Image')         # 3D grayscale
viewer.add_labels(labels_ws, name='Labels')    # 3D segmentation labels
napari.run()

from skimage.measure import regionprops_table
import pandas as pd

props = regionprops_table(labels_ws, properties=['label', 'area', 'centroid'])
df = pd.DataFrame(props)
print(df.head())

In [ ]:
#probar
"""
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
from scipy.ndimage import distance_transform_edt, label
from skimage.filters import threshold_otsu
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage import exposure

# --- Load 3D Volume from TIFF ---
path = "your_image_stack.tif"  # Replace with your converted 3D TIFF file
volume = imread(path)  # shape: (Z, Y, X)
print(f"Loaded volume with shape: {volume.shape}")

# --- Normalize volume ---
volume = exposure.rescale_intensity(volume.astype(np.float32))

# --- Threshold using Otsu ---
thresh = threshold_otsu(volume)
binary = volume > thresh
print(f"Applied Otsu threshold: {thresh:.4f}")

# --- Distance transform ---
distance = distance_transform_edt(binary)

# --- Marker detection ---
coords = peak_local_max(distance, labels=binary, footprint=np.ones((3, 3, 3)), exclude_border=False)
marker_mask = np.zeros_like(distance, dtype=bool)
marker_mask[tuple(coords.T)] = True
markers, _ = label(marker_mask)

# --- Watershed segmentation ---
labels = watershed(-distance, markers, mask=binary)
print(f"Segmented {labels.max()} regions")

# --- Visualize central Z-slice ---
zmid = volume.shape[0] // 2
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(volume[zmid], cmap='gray')
plt.title("Original Z-slice")
plt.subplot(1, 2, 2)
plt.imshow(labels[zmid], cmap='nipy_spectral')
plt.title("Labeled Z-slice")
plt.tight_layout()
plt.show()

# --- View interactively in napari (optional) ---
try:
    import napari
    viewer = napari.Viewer()
    viewer.add_image(volume, name="Original")
    viewer.add_labels(labels, name="3D Segmentation")
    napari.run()
except ImportError:
    print("Install napari with `pip install napari[all]` for 3D interactive view.")
"""
"""
OPTIONAL
import nibabel as nib
nii = nib.load("your_stack.nii.gz")
volume = nii.get_fdata()
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from scipy.ndimage import distance_transform_edt, label
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects
from tifffile import imread, imwrite
from skimage.measure import regionprops_table
import pandas as pd
import imageio

# ==== Load Image ====
img = "C://Users//nahue//Downloads//Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.tif"
volume_raw = imread(img)
print("Raw shape:", volume_raw.shape)

# Select first channel
volume = volume_raw[:, 0, :, :]
print("Volume shape:", volume.shape)

# ==== Preprocess ====
volume = (volume - volume.min()) / (volume.max() - volume.min())
thresh = threshold_otsu(volume)
binary = volume > thresh
binary = remove_small_objects(binary, min_size=200)

# ==== Segmentation ====
distance = distance_transform_edt(binary)
coords = peak_local_max(distance, labels=binary, footprint=np.ones((3, 3, 3)), exclude_border=False)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = label(mask)
labels_ws = watershed(-distance, markers, mask=binary)

# ==== Visualize 6 Slices ====
fig, axes = plt.subplots(4, 3, figsize=(20,12))#2,3
step = max(1, volume.shape[0] // 12)#6
for i, ax in enumerate(axes.flat):
    z = i * step
    ax.imshow(labels_ws[z], cmap="nipy_spectral")
    ax.set_title(f"Z = {z}")
    ax.axis("off")
plt.tight_layout()
plt.show()

# ==== Export segmentation as TIF ====
imwrite("segmentation_labels.tif", labels_ws.astype(np.uint16))
imwrite("intensity_volume.tif", (volume * 255).astype(np.uint8))

# ==== Export animated GIF ====
frames = [(labels_ws[z] * 10).astype(np.uint8) for z in range(volume.shape[0])]
imageio.mimsave("segmentation_stack.gif", frames, duration=0.1)

# ==== Export measurements ====
props = regionprops_table(labels_ws, properties=['label', 'area', 'centroid'])
df = pd.DataFrame(props)
df.to_csv("segmentation_measurements.csv", index=False)
#print("Exported segmentation and region data.")
#print(df.head())